**CNN - Basic convolutional neural networks**

Recommended reads: 
* https://medium.freecodecamp.org/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050
* https://towardsdatascience.com/applied-deep-learning-part-4-convolutional-neural-networks-584bc134c1e2

**Convolutional neural network:**
![img 404](https://cdn-images-1.medium.com/max/1600/1*NQQiyYqJJj4PSYAeWvxutg.png)

CNN usually consists of a series of convolution + pooling layers, followed by fully connected layers

**Convolutions:** <br />
Definitions:
<img src="https://cdn-images-1.medium.com/max/1600/1*cTEp-IvCCUYPTT0QpE3Gjg@2x.png" alt="Drawing" style="width: 400px;"/>

**Input** - input image, usually a 3D array of RGB values. <br />
**Kernel** - filter, matrix operation that will be applied to transform input image <br />

Convolution - cell 1
<img src="https://cdn-images-1.medium.com/max/1600/1*ghaknijNGolaA3DpjvDxfQ@2x.png" alt="Drawing" style="width: 400px;"/>
Convolution - cell 2
<img src="https://cdn-images-1.medium.com/max/1600/1*oxOsZPfZFxgGZw2ycQnenw@2x.png" alt="Drawing" style="width: 400px;"/>


**Pooling:** <br />

<img src="https://cdn-images-1.medium.com/max/1600/1*ReZNSf_Yr7Q1nqegGirsMQ@2x.png" alt="Drawing" style="width: 400px;"/>

MaxPooling takes the maximum value from the selected cells. Main purpose is to downsample the input but keep the most important information


Demo showcase - let's start with the usual - sample photo of a cat!:

In [ ]:
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

# load an image from file
raw_image = load_img('./data/cnn_image/image_cat.jpeg', target_size=(224, 224))
plt.imshow(raw_image)

We are going to use a CNN that has already been trained for us and is publicly available - VGG16. It's a network consisting of 16 layers <a href="https://cdn-images-1.medium.com/max/1600/1*U8uoGoZDs8nwzQE3tOhfkw@2x.png">(see architecture)</a> and has already been pre-trained for us on <a href="http://www.image-net.org/"> Imagenet dataset </a>.

Due to it's popularity VGG16 is already included in some of the most popular ML frameworks - we will use the one  <a href="https://keras.io/applications/#vgg16">provided by Keras</a>.


In [ ]:
from keras.applications.vgg16 import VGG16

model = VGG16(weights = 'imagenet', input_shape=(224, 224, 3))

Now we can convert our image to array of values.

In [ ]:
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import decode_predictions

image_arr = img_to_array(raw_image)
image_arr.shape

Due to API requirements, we need an arary of images, therefore we add 1 dimension

In [ ]:
image = image_arr.reshape((1, image_arr.shape[0], image_arr.shape[1], image_arr.shape[2]))
image.shape

Now we can run the prediction and see the output of our CNN network!

In [ ]:
item = model.predict(image)
labels = decode_predictions(item)
print(labels)

Usually it is advised to preprocess the image before feeding it into vgg16. Keras also provides us with functions for that. Let's see how our image would look like after preprocessing:

In [ ]:
from keras.applications.vgg16 import preprocess_input

image_arr = img_to_array(raw_image)
image = image_arr.reshape((1, image_arr.shape[0], image_arr.shape[1], image_arr.shape[2]))

# prepare the image for the VGG model
image_processed = preprocess_input(image)
plt.imshow(image_processed[0])

Let's see how it would be classified by a network now!

In [ ]:
item = model.predict(image_processed)
labels = decode_predictions(item)
print(labels)

** Part II - CNN Activation ** <br />
Let's try to analize why our network decided to say that the picture is a (persian) cat. We can use <a href="https://raghakot.github.io/keras-vis/">keras-vis</a> module to inspect our vgg16 network.

Great example can be found in attached <a href="https://github.com/raghakot/keras-vis/blob/master/examples/vggnet/activation_maximization.ipynb">keras vis notebooks </a>

In [ ]:
from keras.applications import VGG16
from vis.utils import utils
from keras import activations

# Build the VGG16 network with ImageNet weights
model = VGG16(weights='imagenet', include_top=True)

# Utility to search for layer index by name.
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'predictions')

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

Let's have a look how VGG16 "imagines" Persian cats to look like. We can expore the activation for "Persian cat" class. According to <a href="https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a">this document</a>, Persian cat's ID is 283. We therefore explore what activate's the neuron responsible for class no. 283. 

For that, we use <a href="https://raghakot.github.io/keras-vis/vis.visualization/#visualize_activation">visualize_activation</a> from Keras API. This function will iteratively try to generate the input that will maximize the output of the given layer -> in our case we will be checking which input will maximize the output of a neuron that maximizes the "Persian cat" class.

In [ ]:
from vis.visualization import visualize_activation

CLASS_ID=283

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)

img = visualize_activation(model, layer_idx, filter_indices=CLASS_ID)
plt.imshow(img)

Let's do a couple more iterations on activation function

In [ ]:
img = visualize_activation(model, layer_idx, filter_indices=CLASS_ID, max_iter=500, verbose=True)
plt.imshow(img)

Following <a href="https://raghakot.github.io/keras-vis/visualizations/activation_maximization/">the hints</a> of the library's author, let's try to introduce Jitter.

In [ ]:
from vis.input_modifiers import Jitter


# Jitter 16 pixels along all dimensions to during the optimization process.
img = visualize_activation(model, layer_idx, filter_indices=CLASS_ID, max_iter=500, input_modifiers=[Jitter(16)])
plt.imshow(img)

In the end, let's try to plot heatmap of our CNN and see which part of image triggered it to believe that this is a persian cat. 

For that, we will use the <a href="https://raghakot.github.io/keras-vis/vis.visualization/#visualize_cam">visualize_cam</a> function from Keras API. 

Intuition + papre reference for how it works <a href="https://jacobgil.github.io/deeplearning/class-activation-maps">can be found here</a>

In [ ]:
from vis.visualization import visualize_cam

img = visualize_cam(model, layer_idx, filter_indices=CLASS_ID, seed_input=image_processed)
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(image_processed[0])

**Excercise 1.** Try to upload your own picture and see how would it be recognized by the network! You can use <a href="https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a">Imagenet data</a> to see what classes can you expect!



**Excercise 2.** Pick <a href="https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a">Imagenet class</a> other than cat. Try to see what activates neuron responsible for it.

